In [1]:
import sys
import os
import warnings
import time

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
warnings.filterwarnings("ignore")

In [2]:
from src.database.db_connection import *
from src.database.preparation import *
from src.utilities.step1_simplify_fds import *
from src.utilities.step2_clustering_and_generating_repairs import *
from src.utilities.step3_convert_back_to_db import *
from src.utilities.step4_group_similar_sentences import *
from src.evaluation.metrics import *
from src.database.db_connection import *

In [3]:
file_path = os.path.join(
    os.getcwd(), "..", "..", "..",
    "queries", "FP", "synthetic", "Students.csv"
)
file_path = os.path.abspath(file_path)
df_data = load_csv_to_df(file_path)

In [4]:
engine = create_engine_for_db()

In [5]:
df_data.to_sql('students_truth',engine,schema='prototype_fp',index=False,if_exists='replace')

10

In [6]:
fd_constraints = [(['sid', 'university'],['sname', 'address', 'postcode', 'emailid', 'phonenum','age'])]

In [7]:
fd_LHS, fd_RHS = fd_constraints[0]

In [8]:
fd_LHS

['sid', 'university']

In [9]:
fd_RHS

['sname', 'address', 'postcode', 'emailid', 'phonenum', 'age']

In [10]:
fd_constraints = simplify_FDs(fd_constraints)

In [11]:
fd_constraints

[(['sid', 'university'], ['sname']),
 (['sid', 'university'], ['address']),
 (['sid', 'university'], ['postcode']),
 (['sid', 'university'], ['emailid']),
 (['sid', 'university'], ['phonenum']),
 (['sid', 'university'], ['age'])]

In [12]:
# violation_levels = [{'target_violation_fraction':0.2, 'dups_per_cluster':(1,2), 'cells_corrupted':(1,2)},
# {'target_violation_fraction':0.5, 'dups_per_cluster':(2,3), 'cells_corrupted':(2,3)},
# {'target_violation_fraction':0.8, 'dups_per_cluster':(3,5), 'cells_corrupted':(3,5)},
# {'target_violation_fraction':1.0, 'dups_per_cluster':(2,3), 'cells_corrupted':(2,3)}]

violation_levels = [{'target_violation_fraction':0.5, 'dups_per_cluster':(2,3), 'cells_corrupted':(2,3)}]
# {'target_violation_fraction':0.5, 'dups_per_cluster':(2,3), 'cells_corrupted':(1,2)},
# {'target_violation_fraction':0.8, 'dups_per_cluster':(3,5), 'cells_corrupted':(2,3)},
# {'target_violation_fraction':1.0, 'dups_per_cluster':(6,10), 'cells_corrupted':(3,5)}]

num_inconsistent_relations = 2

delete_cost = 2
update_cost = 1
eps=1e-9
prob_type = 'UNIFORM'
constraint_hardness = [1.0] * len(fd_constraints)

query_file_name_expected_Prob = "../../../queries/FP/synthetic/Students_truth.sql"
query_file_name_actual_Prob = "../../../queries/FP/synthetic/Students_prob.sql"

In [13]:
for j,violation_level in enumerate(violation_levels,start=1):
    total_time = 0.0
    total_precision_prob, total_recall_prob,total_coverage_prob, total_noise_prob = 0.0, 0.0, 0.0, 0.0
    total_rvs = 0
    max_repair_actions_possible = 0
    min_cost_actions_suggested = 0
    target_violation_fraction = violation_level['target_violation_fraction']
    dups_per_cluster = violation_level['dups_per_cluster']
    cells_corrupted = violation_level['cells_corrupted']

    inconsistent_dfs = generate_corruptions_for_student_data(df_data, fd_LHS, fd_RHS, num_inconsistent_relations, target_violation_frac=target_violation_fraction, 
                                          dups_per_cluster=dups_per_cluster, cells_corrupted=cells_corrupted)
    for (i,inconsistent_df) in enumerate(inconsistent_dfs,start=1):

        with engine.begin() as connection:
            connection.execute(text("""
            TRUNCATE TABLE prototype_fp.students_prob
            RESTART IDENTITY
            CASCADE;
            """))
            connection.execute(text("""
            UPDATE prototype_fp."_dict"
            SET dict = dictionary('')
            WHERE name = :name;
            """), {"name": "students_dict"})
        start = time.perf_counter()
        inconsistent_df = preprocess_data(inconsistent_df)
        table_name = f"students_{j}_{i}" 
        inconsistent_df.to_sql(table_name,engine,schema='prototype_fp',index=False,if_exists='replace')
        actions,cluster_ids,rv_probs = compute_actions(inconsistent_df, fd_constraints, constraint_hardness,delete_cost, update_cost,eps,prob_type)
        print(actions[0])
        all_actions,total_repairs_possible = min_cost_actions(cluster_ids,actions,delete_cost,update_cost)
        print(total_repairs_possible)
        max_repair_actions_possible += total_repairs_possible
        min_cost_actions_suggested += len(all_actions)
        repaired_versions =  apply_combined_actions(inconsistent_df, all_actions, uuid_col="uuid")
        repaired_versions = pd.concat(repaired_versions, ignore_index=True)
        repaired_versions = repaired_versions.drop(columns=["uuid"])
        rv_probabilities = filter_and_condition_min_cost_rvs(rv_probs, repaired_versions)
        repaired_df = group_similar_rows_together(repaired_versions)

        repaired_df.to_sql('students_prob',engine,schema='prototype_fp',index=False,if_exists='append')
        update_stmt_dic = f"UPDATE prototype_fp._dict set  dict = dictionary('') where name='students_dict';"
        with engine.begin() as connection:
            connection.execute(text(update_stmt_dic))
        entries = [f"{k}:{v}" for k, v in rv_probabilities.items()]
        joined = ';'.join(entries)

        update_stmt = f"UPDATE prototype_fp._dict\nSET dict=add(dict, '{joined}')\nWHERE name='students_dict';"
        with engine.begin() as connection:
            connection.execute(text(update_stmt))
        elapsed = time.perf_counter() - start
        total_rvs += len(rv_probabilities)
        total_time += elapsed

        results_Prob,results_with_truth_set_prob = calculate_expected_values(query_file_name_expected_Prob, query_file_name_actual_Prob, engine)
        precision_prob, recall_prob,coverage_prob, noise_prob = summarize_results(results_Prob)
        
        print(f"\nViolation Level {j}: Processed inconsistent relation {i}")
        print(f"Precision Probabilistic: {precision_prob:.4f}")
        print(f"Recall Probabilistic: {recall_prob:.4f}")
        print(f"Coverage Probabilistic: {coverage_prob:.4f}")
        print(f"Noise Probabilistic: {noise_prob:.4f}")
        total_precision_prob += precision_prob
        total_recall_prob += recall_prob
        total_coverage_prob += coverage_prob
        total_noise_prob += noise_prob
    
    avg_time_violation_level = total_time/len(inconsistent_dfs)
    avg_number_of_rvs = total_rvs/len(inconsistent_dfs)
    avg_max_repair_actions_possible = max_repair_actions_possible/len(inconsistent_dfs)
    avg_min_cost_actions_suggested = min_cost_actions_suggested/len(inconsistent_dfs)
    avg_precision_prob = total_precision_prob / len(inconsistent_dfs)
    avg_recall_prob = total_recall_prob / len(inconsistent_dfs)
    avg_coverage_prob = total_coverage_prob / len(inconsistent_dfs)
    avg_noise_prob = total_noise_prob / len(inconsistent_dfs)

    
    print(f"\nViolation Level {j}: Average time taken {avg_time_violation_level}")
    print(f"Violation Level {j}: Average number of RVs used {int(avg_number_of_rvs)}")
    print(f"Violation Level {j}: Max possible repair actions {int(avg_max_repair_actions_possible)}")
    print(f"Violation Level {j}: Min cost repair actions suggested {int(avg_min_cost_actions_suggested)}")
    print(f"Average Precision Probabilistic for violation level {j}: {avg_precision_prob:.4f}")
    print(f"Average Recall Probabilistic for violation level {j}: {avg_recall_prob:.4f}")
    print(f"Average Coverage Probabilistic for violation level {j}: {avg_coverage_prob:.4f}")
    print(f"Average Noise Probabilistic for violation level {j}: {avg_noise_prob:.4f}")

[{'FD': 1, 'LHS': ['sid', 'university'], 'Cluster': 4, 'Tuples': {4}, 'TargetValue': 'Claire Stevens', 'DR': [18, 7], 'UR': [], 'Cost': 4, 'min_cost': 1, 'rv': 'fd1sname4=1'}, {'FD': 1, 'LHS': ['sid', 'university'], 'Cluster': 4, 'Tuples': {18, 4}, 'TargetValue': 'Claire Stevens', 'DR': [7], 'UR': [], 'Cost': 2, 'min_cost': 1, 'rv': 'fd1sname4=2'}, {'FD': 1, 'LHS': ['sid', 'university'], 'Cluster': 4, 'Tuples': {4, 7}, 'TargetValue': 'Claire Stevens', 'DR': [18], 'UR': ['7:sname=Claire Stevens'], 'Cost': 3, 'min_cost': 1, 'rv': 'fd1sname4=3'}, {'FD': 1, 'LHS': ['sid', 'university'], 'Cluster': 4, 'Tuples': {18, 4, 7}, 'TargetValue': 'Claire Stevens', 'DR': [], 'UR': ['7:sname=Claire Stevens'], 'Cost': 1, 'min_cost': 1, 'rv': 'fd1sname4=4'}, {'FD': 1, 'LHS': ['sid', 'university'], 'Cluster': 4, 'Tuples': {7}, 'TargetValue': 'Claore Stevens', 'DR': [18, 4], 'UR': [], 'Cost': 4, 'min_cost': 1, 'rv': 'fd1sname4=5'}, {'FD': 1, 'LHS': ['sid', 'university'], 'Cluster': 4, 'Tuples': {18, 7}, '